In [1]:
import tensorflow as tf

from src.swins.models import SwinTransformer

ImportError: cannot import name 'SwinTransformerBlock' from 'src.swins.blocks.swin_transformer_block' (/workspaces/Object-Detection-BB/src/swins/blocks/swin_transformer_block.py)